In [1]:
# importing all dependencies
import pandas as pd
import requests
import numpy as np
from citipy import citipy

# importing api key
from config import weather_api_key

In [2]:
# creating set of random latitude and longitude numbers.
lats = np.random.uniform(-90, 90, size=2000)
lngs = np.random.uniform(-180, 180, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

# creating a list for coordinates
coordinates = list(lat_lngs)

In [3]:
# creating list of cities with the nearest city for each coordinates
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # if city unique, add city to list
    if city not in cities:
        cities.append(city)
# calculate number of cities
len(cities)

753

In [4]:
# weather map API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# testing url to extract requested data
test_url = requests.get(url + "&q=" + "Boston").json()
test_url

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 34.95,
  'feels_like': 24.98,
  'temp_min': 33.8,
  'temp_max': 36,
  'pressure': 997,
  'humidity': 75},
 'visibility': 10000,
 'wind': {'speed': 10.36, 'deg': 310},
 'clouds': {'all': 90},
 'dt': 1612383120,
 'sys': {'type': 1,
  'id': 3486,
  'country': 'US',
  'sunrise': 1612353329,
  'sunset': 1612389639},
 'timezone': -18000,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [5]:
# creating list to hold weather data
city_data = []

# validating counters.
record_count = 1
set_count = 1

# looping through cities
for i, city in enumerate(cities):

    # grouping cities in set of 50 for logging
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # creating endpoint url including cities
    city_url = url + "&q=" + city.replace(" ","+")

# running API call for each city
    try:
        # parsing json data and pulling request
        city_weather = requests.get(city_url).json()
        # parsing out needed data only
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
     
        # adding city details into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [6]:
len(city_data)

693

In [10]:
# converting arrays to dataframe
city_data_df = pd.DataFrame(city_data)
# reordering columns
new_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[new_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mahebourg,MU,-20.4081,57.7000,78.80,89,75,4.61,moderate rain
1,Castro,BR,-24.7911,-50.0119,80.85,70,12,4.05,few clouds
2,Qaanaaq,GL,77.4840,-69.3632,-21.50,68,73,2.80,broken clouds
3,Bonavista,CA,48.6499,-53.1147,37.00,100,100,4.61,light rain
4,Mataura,NZ,-46.1927,168.8643,51.78,67,54,5.26,broken clouds
5,Ribeira Grande,PT,38.5167,-28.7000,53.60,66,40,26.46,scattered clouds
6,Kapaa,US,22.0752,-159.3190,68.00,88,75,8.05,broken clouds
7,Saint-Philippe,RE,-21.3585,55.7679,80.60,74,0,2.30,clear sky
8,Hermanus,ZA,-34.4187,19.2345,64.00,86,4,6.13,clear sky
9,Hobart,AU,-42.8794,147.3294,57.99,100,90,4.61,light intensity drizzle


In [11]:
# saving dataframe in csv file
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")